In [9]:
# import packages
import cv2
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import load_img, img_to_array
import numpy as np
 
#load model # Accuracy=97.4 , validation Accuracy = 99.1 # very light model, size =5MB
model = load_model('D://Hitesh Data Scientist//Computer Vision//OpenCV//Helmet Detection//helmet_detection_model.h5') # cnn
 
# model accept below hight and width of the image
img_width, img_hight = 200, 200
 

In [8]:
# Load the Cascade face Classifier
face_cascade = cv2.CascadeClassifier("D://Hitesh Data Scientist//Computer Vision//OpenCV//Helmet Detection//haarcascade_frontalface_default.xml")
 
#startt  web cam
cap = cv2.VideoCapture(0) # for webcam
#cap = cv2.VideoCapture('videos/Mask - 34775.mp4') # for video
 
img_count_full = 0
 
#parameters for text
# font 
font = cv2.FONT_HERSHEY_SIMPLEX 
# org 
org = (1, 1)
class_lable=' '      
# fontScale 
fontScale = 1 #0.5
# Blue color in BGR 
color = (255, 0, 0) 
# Line thickness of 2 px 
thickness = 2 #1
 
#sart reading images and prediction
while True:
    img_count_full += 1
     
    #read image from webcam
    responce, color_img = cap.read()
    #color_img = cv2.imread('image1.jpg')
     
    #if respoce False the break the loop
    if responce == False:
        break    
     
    # Convert to grayscale
    gray_img = cv2.cvtColor(color_img, cv2.COLOR_BGR2GRAY)
     
    # Detect the faces
    faces = face_cascade.detectMultiScale(gray_img, 1.1, 6) # 1.3, 6) for 1.mp4
     
    #take face then predict class mask or not mask then draw recrangle and text then display image
    img_count = 0
    for (x, y, w, h) in faces:
        org = (x-10,y-10)
        img_count +=1 
        color_face = color_img[y:y+h,x:x+w] # color face
        cv2.imwrite('D://Hitesh Data Scientist//Computer Vision//OpenCV//Helmet Detection//Faces//input//%d%dface.jpg'%(img_count_full,img_count),color_face)
        img = load_img('D://Hitesh Data Scientist//Computer Vision//OpenCV//Helmet Detection//Faces//input//%d%dface.jpg'%(img_count_full,img_count), target_size=(img_width,img_hight))
         
        img = img_to_array(img)/255
        img = np.expand_dims(img,axis=0)
        pred_prob = model.predict(img)
        #print(pred_prob[0][0].round(2))
        pred=np.argmax(pred_prob)
             
        if pred==0:
            print("User with Helmet - predic = ",pred_prob[0][0])
            class_lable = "Wearing Helmet You are safe."
            color = (0, 255, 0)
            cv2.imwrite('D://Hitesh Data Scientist//Computer Vision//OpenCV//Face Mask Detection//Faces//with_mask//%d%dface.jpg'%(img_count_full,img_count),color_face)
                  
        else:
            print('user not wearing Helmet - prob = ',pred_prob[0][1])
            class_lable = "Wearing No Helmet Not safe"
            color = (255, 0, 0)
            cv2.imwrite('D://Hitesh Data Scientist//Computer Vision//OpenCV//Helmet Detection//Faces//without_helmet//%d%dface.jpg'%(img_count_full,img_count),color_face)
                 
        cv2.rectangle(color_img, (x, y), (x+w, y+h), (0, 0, 255), 3)
        # Using cv2.putText() method 
        cv2.putText(color_img, class_lable, org, font,  
                                   fontScale, color, thickness, cv2.LINE_AA) 
     
    # display image
    cv2.imshow('LIVE face mask detection', color_img)
     
    if cv2.waitKey(1) &0xFF == ord('q'):
        break
 
# Release the VideoCapture object
cap.release()
cv2.destroyAllWindows()